In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
train = pd.read_csv('train.csv') # /kaggle/input/nlp-getting-started/train.csv
test = pd.read_csv('test.csv') # /kaggle/input/nlp-getting-started/test.csv
#sample_submission = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')

In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [5]:
train['text'].sample(30)

2818    .POTUS #StrategicPatience is a strategy for #G...
5795    @jasalhad @brianboru67 @Jimskiv92 @hijinks1967...
3375    Bend Post Office roofers cut gas line prompt e...
352     Seeing that army of whitewalkers was the very ...
5184                                          Obliterated
2660    Apollo Brown - 'Detonate' f. M.O.P. http://t.c...
4530    And you wonder why he's injured every year htt...
4539    Top Stories - Google 4 dead dozens injured in ...
5951                     *CUE THE JARIANA STANS SCREAMING
7290    MY GIRL GOT A GIRLFRIEND CHEVY BLUE LIKE WHIRL...
2453    @greateranglia I know the cow incident not yr ...
1193    A Marshall Plan for the United States by Dambi...
2400    Sound judgement by MPC - premature rises could...
6301    @Stretcher @Rexyy @invalid @Towel let's have b...
2160    As of 2010 there were 17 Beluga deaths reporte...
5849    fresh out da shower lookss ?? (still loving th...
6504    10 Ways To Survive and Escape Martial Law | Wo...
284     RT: fi

In [6]:
train['keyword'].isnull().values.sum()

61

In [7]:
train['location'].isnull().values.sum()

2533

In [8]:
train['text'].isnull().values.any()

False

In [9]:
# Drop rows that have missing keyword
##
train = train.dropna(axis = 0,subset=['keyword'])

In [10]:
train.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


**Text processing require:**  
- remove punctuation  
- remove numbers  
- to lowercase  
- remove stopwords  
- lemmatize  
- remove non-english words  

**NOTE** : The order of the steps matter. The text processing function should execute steps in this outlined order.

In [11]:
# A function to process text as outlined above

import re
import string
import nltk
from nltk.corpus import stopwords

#words = nltk.download('words')
words = set(nltk.corpus.words.words())

regex = re.compile('[^a-zA-Z]') # remove emojis and any non-letters

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def text_process(text):
    cln = [c for c in text if c not in string.punctuation]
    cln = ''.join(cln)
    cln = [c for c in cln if not c.isdigit()]
    cln = ''.join(cln)
    cln = cln.lower()
    cln = [word for word in cln.split() if word not in stopwords.words('english')]
    cln = ' '.join(cln)
    cln = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(cln)]
    cln = ' '.join(cln)
    cln = regex.sub(' ', cln)
    cln = [word for word in nltk.wordpunct_tokenize(cln) if word in words or not word.isalpha()]
    return cln #[word for word in cln.split()]

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

bow_transformer = CountVectorizer(analyzer=text_process)
bow_transformer.fit(train['text'])

CountVectorizer(analyzer=<function text_process at 0x7f7a724dc710>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [21]:
len(bow_transformer.vocabulary_)

6693

In [22]:
print(bow_transformer.transform([train['text'][1230]]))

  (0, 327)	1
  (0, 1049)	1
  (0, 2100)	1
  (0, 2249)	1
  (0, 2699)	1
  (0, 3019)	1


In [23]:
bow_transformer.get_feature_names()[1048]

'civilization'

In [24]:
bow_text = bow_transformer.transform(train['text'])

In [25]:
bow_text.shape

(7552, 6693)

In [26]:
#tfidf

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(bow_text)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [27]:
print(tfidf_transformer.transform(bow_transformer.transform([train['text'][1230]])))

  (0, 3019)	0.4700546547381698
  (0, 2699)	0.34618951012493654
  (0, 2249)	0.22878569166423807
  (0, 2100)	0.43926482529741256
  (0, 1049)	0.4700546547381698
  (0, 327)	0.43926482529741256


In [28]:
tfidf_transformer.idf_[bow_transformer.vocabulary_['cladding']]

8.543405753193557

In [29]:
tfidf_text = tfidf_transformer.transform(bow_text)

In [30]:
tfidf_text.shape

(7552, 6693)

**ML Algorithm Selection**

In [31]:
train['target'].value_counts()

0    4323
1    3229
Name: target, dtype: int64

In [32]:
# So, a classification algorithm eg. Logistic regression, naive bayes

In [34]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_text, train['target'], test_size=0.3, random_state=101)

In [36]:
from sklearn.metrics import confusion_matrix, classification_report

In [37]:
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

nb.fit(X_train, y_train)
nb_pred = nb.predict(X_test)

rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)

/home/lyrax/.local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/lyrax/.local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [38]:
print("---------------------------------")
print("Logistic Regression")
print("---------------------------------")
print(confusion_matrix(y_test, lr_pred))
print(classification_report(y_test, lr_pred))
print("---------------------------------")
print("Naive Bayes MultinomialNB")
print("---------------------------------")
print(confusion_matrix(y_test, nb_pred))
print(classification_report(y_test, nb_pred))
print("---------------------------------")
print("RandomForestClassifier")
print("---------------------------------")
print(confusion_matrix(y_test, rfc_pred))
print(classification_report(y_test, rfc_pred))

---------------------------------
Logistic Regression
---------------------------------
[[1162  165]
 [ 299  640]]
              precision    recall  f1-score   support

           0       0.80      0.88      0.83      1327
           1       0.80      0.68      0.73       939

    accuracy                           0.80      2266
   macro avg       0.80      0.78      0.78      2266
weighted avg       0.80      0.80      0.79      2266

---------------------------------
Naive Bayes MultinomialNB
---------------------------------
[[1136  191]
 [ 299  640]]
              precision    recall  f1-score   support

           0       0.79      0.86      0.82      1327
           1       0.77      0.68      0.72       939

    accuracy                           0.78      2266
   macro avg       0.78      0.77      0.77      2266
weighted avg       0.78      0.78      0.78      2266

---------------------------------
RandomForestClassifier
---------------------------------
[[1089  238]
 [ 311

In [39]:
# Logistic regression did quite well than the other two

In [40]:
# break()
#stop

In [42]:
# Adding XgbClassifier
##
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

xgb = xgb.XGBClassifier()

param_xgb = dict(
    silent=[False, True],
    learning_rate=[0.01, 0.005, 0.07],
    subsample = [0.5, 0.7],
    objective=['binary:logistic'], 
    n_estimators=[500, 1000], 
    max_depth=[6, 10, 12], 
    gamma=[0, 10, 30]
)

grid_xgb = GridSearchCV(estimator=xgb, param_grid=param_xgb, cv=5, n_jobs=-1, verbose=10)

grid_xgb.fit(X_train, y_train)
print("Best score: ", grid_xgb.best_score_)
print("Best parameters: ", grid_xgb.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   47.2s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 11.9min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed: 17.7min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed: 18.7min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed: 22

Best score:  0.7633371169125993
Best parameters:  {'gamma': 0, 'learning_rate': 0.07, 'max_depth': 12, 'n_estimators': 500, 'objective': 'binary:logistic', 'silent': False, 'subsample': 0.7}


In [ ]:
# best parameters for xgbc: 'gamma': 0, 'learning_rate': 0.07, 'max_depth': 12, 'n_estimators': 500,
# 'objective': 'binary:logistic', 'silent': False, 'subsample': 0.7

In [43]:
# Predictions
##
pred_xgb = grid_xgb.predict(X_test)

# Evaluation
print(confusion_matrix(y_test, pred_xgb))
print("\n")
print(classification_report(y_test, pred_xgb))

[[1114  213]
 [ 303  636]]


              precision    recall  f1-score   support

           0       0.79      0.84      0.81      1327
           1       0.75      0.68      0.71       939

    accuracy                           0.77      2266
   macro avg       0.77      0.76      0.76      2266
weighted avg       0.77      0.77      0.77      2266



**Building a data pipeline to predict on the test data.**

In [48]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

pipe = Pipeline([
    ("bow cv", CountVectorizer(analyzer=text_process)),
    ("tfidf", TfidfTransformer()),
    ("xgbc", XGBClassifier(gamma=0, learning_rate=0.07, max_depth=12, n_estimators=500,
                               objective='binary:logistic', silent=False, subsample=0.7)) # ("lr classifier", LogisticRegression())
])

In [49]:
text_train, text_test, label_train, label_test = train_test_split(train['text'], train['target'],
                                                                  test_size=0.3, random_state=101) 

In [50]:
pipe.fit(text_train, label_train)

Pipeline(memory=None,
         steps=[('bow cv',
                 CountVectorizer(analyzer=<function text_process at 0x7f7a724dc710>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\...
                               interaction_constraints=None, learning_rate=0.07,
                               max_delta_step=0, max_depth=12,
                               min_child_weight=1, missing=nan,
                               monotone_constraints=None, n_estimators=500,
                               n_jobs=0, num_parallel_tree=1,
                     

In [51]:
pipe_pred = pipe.predict(test['text'])

In [52]:
pipe_pred.shape

(3263,)

In [53]:
test.shape

(3263, 4)

In [54]:
submission2 = pd.DataFrame({
    "id": test['id'],
    "target": pipe_pred
})

In [55]:
submission2

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [56]:
#submission2.to_csv('submission2.csv', index=False)